In [ ]:
!pip install ../input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/ > /dev/null # no output
package_path = '../input/unetmodelscript' # add unet script dataset
import sys
sys.path.append(package_path)
from model import Unet # import Unet model from the script

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
from tqdm.notebook import tqdm_notebook as tqdm
import seaborn as sns
import albumentations  as albu
from albumentations.pytorch import ToTensor
import random

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision
from torchvision import models
from torch.autograd import Function

In [ ]:
# 乱数のシードを設定
seed = 1234
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
save_dir_weights  = "./weights"
save_dir_logs  = "./logs"

if os.path.exists(save_dir_weights) == False:
    os.makedirs(save_dir_weights)
if os.path.exists(save_dir_logs) == False:
    os.makedirs(save_dir_logs)

# データ確認

## 画像データ

In [ ]:
input_dir = "../input/severstal-steel-defect-detection/"
input_dir_Train  = os.path.join(input_dir, 'train_images')
input_dir_Test  = os.path.join(input_dir, 'test_images')
filelist_Train = os.listdir(input_dir_Train)
filelist_Test = os.listdir(input_dir_Test)
print('train data size : {}'.format(len(filelist_Train)))
print('test data size : {}'.format(len(filelist_Test)))

In [ ]:
index = 0
path = os.path.join(input_dir_Train, filelist_Train[0])
image = cv2.imread(path)

In [ ]:
plt.imshow(image)

In [ ]:
print('image shape : {}'.format(image.shape))

## csvファイル

In [ ]:
df_path = os.path.join(input_dir, 'train.csv')
df = pd.read_csv(df_path)
df.head()

In [ ]:
print('defect num : {}'.format(df['ImageId'].nunique()))
print('no defect num : {}'.format(len(filelist_Train) - df['ImageId'].nunique()))

In [ ]:
defect_class = np.zeros((4))
for i in tqdm(range(len(df))):
    class_id = df.iloc[i]['ClassId']
    defect_class[class_id - 1] += 1

In [ ]:
fig, ax = plt.subplots()
sns.barplot(x=np.arange(1, 5), y=defect_class, ax=ax)
ax.set_title("the number of images for each class")
ax.set_xlabel("class")

# マスク画像作成

In [ ]:
def make_df(df):
    df = df.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')
    df['defects'] = df.count(axis=1)
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["defects"], random_state=seed)
    return train_df, val_df

In [ ]:
train_df, val_df = make_df(df)

In [ ]:
def make_mask(index, df):
    filename = df.iloc[index].name
    labels = df.iloc[index, :4]
    masks = np.zeros((256, 1600, 4), dtype=np.float32)
    for idx, label in enumerate(labels):
        if label is not np.nan:
            mask = np.zeros((256*1600), dtype=np.uint8)
            pixels = label.split(' ')
            pixels = [pixels[i:i+2] for i in range(0, len(pixels), 2)]
            for pixel in pixels:
                pos, le = pixel
                pos, le = int(pos), int(le)
                mask[pos-1:pos+le-1] = 1
            masks[:,:,idx] = mask.reshape(256, 1600, order = 'F')
    return filename, masks

In [ ]:
f, m = make_mask(0, train_df)
print('file name : {}'.format(f))
plt.imshow(cv2.imread(os.path.join(input_dir_Train, f)))
plt.show()
plt.imshow(m[:,:,2])
plt.show()

# Data Augmentation

In [ ]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
def get_augmentation(mean, std, phase):
    
    if phase == 'train':
        transform = [
            albu.HorizontalFlip(p=0.5),
            albu.VerticalFlip(p=0.5),
            albu.Resize(256, 256, interpolation=cv2.INTER_NEAREST, p=1),
            albu.Normalize(mean=mean, std=std, p=1),
            ToTensor(),
        ]
    else:
        transform = [
            albu.Resize(256, 256, interpolation=cv2.INTER_NEAREST, p=1),
            albu.Normalize(mean=mean, std=std, p=1),
            ToTensor(),
        ]
    
    return albu.Compose(transform)

# DataLoader作成

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, df, input_dir, phase):
        self.df = df
        self.input_dir = input_dir
        self.transforms = get_augmentation(mean, std, phase) 
        self.phase = phase
    def __getitem__(self, idx):
        filename, mask = make_mask(idx, self.df)
        image = cv2.imread(os.path.join(self.input_dir, filename))
        augmented = self.transforms(image=image, mask=mask)
        image, mask = augmented['image'], augmented['mask']
        mask = mask[0].permute(2, 0, 1)
        return image, mask
    def __len__(self):
        return len(self.df)

In [ ]:
train_dataset = MyDataset(train_df, input_dir_Train, phase = 'train')
val_dataset = MyDataset(val_df, input_dir_Train, phase = 'val')

# 動作確認
index = 0
image, mask = train_dataset.__getitem__(index) 
print(image.size())
plt.imshow(image.to('cpu').detach().numpy().copy()[0])
plt.show()
print(mask.size())
plt.imshow(mask.to('cpu').detach().numpy().copy()[2])
plt.show()

In [ ]:
batch_size = 4

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=6)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=6)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 動作確認
batch_iterator = iter(dataloaders_dict["train"])  # イテレータに変換
inputs, labels = next(batch_iterator)  # 1番目の要素を取り出す
print('inputs size : {}'.format(inputs.size()))
print('labels size : {}'.format(labels.size()))

# ネットワークモデル作成

In [ ]:
!mkdir -p /tmp/.cache/torch/checkpoints/
!cp ../input/resnet18/resnet18.pth /tmp/.cache/torch/checkpoints/resnet18-5c106cde.pth

In [ ]:
model = Unet("resnet18", encoder_weights="imagenet", classes=4, activation=None)

model.train()

print('ネットワーク設定完了：訓練モードに設定しました')

# 損失関数を定義

In [ ]:
criterion = nn.BCEWithLogitsLoss()

# 最適化手法を決定

In [ ]:
optimizer = optim.Adam(
    model.parameters(), lr=5e-4
)

# 学習・検証を実施

In [ ]:
def dice_coeff(pred, mask):
    with torch.no_grad():
        batch_size = len(pred)
        pred = pred.view(batch_size, -1) # Flatten
        mask = mask.view(batch_size, -1)  # Flatten
        pred = (pred>0.5).float()
        mask = (mask>0.5).float()
        smooth = 0.0001
        intersection = (pred * mask).sum()
        dice_pos = (2. * intersection + smooth) / (pred.sum() + mask.sum() + smooth) 
        intersection = ((pred + mask) == 0).sum()
        dice_neg = (2. * intersection + smooth) / ((pred == 0).sum() + (mask == 0).sum() + smooth)
        dice = (dice_pos + dice_neg) / 2.0
        return dice.item()

In [ ]:
def train_model(model, dataloaders_dict, num_epoch, optimizer, criterion, train_loss, train_acc, val_loss, val_acc):
    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    model = model.to(device)

    # ネットワークがある程度固定であれば、高速化させる    
    torch.backends.cudnn.benchmark = True
    
    num_train_imgs = len(dataloaders_dict['train'].dataset)
    num_val_imgs = len(dataloaders_dict['val'].dataset)
    batch_size = dataloaders_dict['train'].batch_size
    
    for epoch in range(num_epoch):
    
        print('Epoch {}/{}'.format(epoch+1, num_epoch))
        print('-------------')
    
        #---- Train section
        epoch_loss = 0.0
        epoch_acc = 0.0
        for img, mask in tqdm(dataloaders_dict['train']):
        
            model.train()

            # GPUが使えるならGPUにデータを送る   
            img = img.to(device) 
            mask = mask.to(device)

            # optimizerを初期化
            optimizer.zero_grad()
        
            output = model(img)
            loss =  criterion(output, mask)
        
            # 訓練時はバックプロパゲーション
            loss.backward()
            optimizer.step()
        
            # 結果の計算
            epoch_loss += loss.item() # lossの合計を更新

            prob = torch.sigmoid(output)
            prob = prob.to('cpu').detach()
            mask = mask.to('cpu').detach()
        
            # ダイス係数の合計を更新
            epoch_acc += dice_coeff(prob, mask)
        
        train_loss.append(epoch_loss / num_train_imgs * batch_size)
        train_acc.append(epoch_acc / num_train_imgs * batch_size)
        
        print('Train {} finished'.format(epoch + 1))
        print('Loss : {}'.format(epoch_loss / num_train_imgs * batch_size))
        print('Accuracy : {}'.format(epoch_acc / num_train_imgs * batch_size))
    
        #---- Val section
        epoch_loss = 0.0
        epoch_acc = 0.0
        with torch.no_grad():
            for img, mask in tqdm(dataloaders_dict['val']):
            
                model.eval()
            
                # GPUが使えるならGPUにデータを送る
                img = img.to(device)
                mask = mask.to(device)

                output = model(img)
                loss = criterion(output, mask)
                
                # 結果の計算
                epoch_loss += loss.item() # lossの合計を更新

                prob = torch.sigmoid(output)
                prob = prob.to('cpu').detach()
                mask = mask.to('cpu').detach()
            
                # ダイス係数の合計を更新
                epoch_acc += dice_coeff(prob, mask)
            
        val_loss.append(epoch_loss / num_val_imgs * batch_size)
        val_acc.append(epoch_acc / num_val_imgs * batch_size)
        
        print('Valid {} finished'.format(epoch + 1))
        print('Loss : {}'.format(epoch_loss / num_val_imgs * batch_size))
        print('Accuracy : {}'.format(epoch_acc / num_val_imgs * batch_size))
        
        torch.save(model.state_dict(), '{}CP{}.pth'.format('./weights/resnet18_', epoch + 1))
        print('Checkpoint {} saved'.format(epoch + 1)) 


In [ ]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []
num_epoch = 20
train_model(model, dataloaders_dict, num_epoch, optimizer, criterion, train_loss, train_acc, val_loss, val_acc)

# 可視化

In [ ]:
plt.title('Resnet18')
plt.xlabel("Epoch")
plt.plot(list(range(num_epoch)), train_acc, val_acc)
plt.ylabel("Acc")
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.tight_layout()
plt.grid(True)
plt.savefig("./logs/acc.png")

In [ ]:
plt.title('Resnet18')
plt.xlabel("Epoch")
plt.plot(list(range(num_epoch)), train_loss, val_loss)
plt.ylabel("Loss")
plt.legend(['Training Loss', 'Validation Loss'])
plt.tight_layout()
plt.grid(True)
plt.savefig("./logs/loss.png")